# Quelle est la page précédente des visiteurs des pages info licenciement

**Définition** : Ce que je veux faire

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [21]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict
tqdm.pandas()


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Si on veut lire en requêtant Elastic Search

In [2]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
  "query": {
    "bool": { 
      "must": [
        {
          "range": {
            "logfile": {
              "gte": "2023-02-01",
              "lt": "2023-04-01"
            }
          }
        }
      ]
    }
  }
}

connection with ElasticSearch established with success


In [3]:
logs = es_connector.execute_query(QUERY, "logs-new")

  0%|          | 0/9365155 [00:00<?, ?it/s]

In [4]:
logs_save = logs.copy()
logs_save.to_csv('../../save_fev_marc_data.csv', index=False)

In [5]:
logs.shape

(9365155, 27)

In [6]:
logs.head()

,timeSpent,timestamp,url,idVisit,lastActionDateTime,lastActionTimestamp,referrerName,referrerTypeName,serverTimePretty,logfile,uvi,type,outil,outilAction,outilEvent,feedbackType,visited,ccAction,query,idCc,recoSelection,recoType,cc,resultSelection,suggestionPrefix,suggestionSelection,referrerUrl
0,0.0,1678089600,https://code.travail.gouv.fr/outils/indemnite-...,33572723,2023-03-05 23:00:00,1678057200,service-public.fr,Websites,00:00:00,2023-03-05,-776740165,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1678089140,https://code.travail.gouv.fr/outils/indemnite-...,33572562,2023-03-05 22:59:59,1678057199,service-public.fr,Websites,23:59:59,2023-03-05,372438899,outil,Indemnité de licenciement,view_step,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,376.0,1678089141,https://code.travail.gouv.fr/outils/indemnite-...,33572562,2023-03-05 22:59:59,1678057199,service-public.fr,Websites,23:59:59,2023-03-05,372438899,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1678089141,https://code.travail.gouv.fr/outils/indemnite-...,33572562,2023-03-05 22:59:59,1678057199,service-public.fr,Websites,23:59:59,2023-03-05,372438899,outil,Indemnité de licenciement,view_step,contrat_travail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1678089153,https://code.travail.gouv.fr/outils/indemnite-...,33572562,2023-03-05 22:59:59,1678057199,service-public.fr,Websites,23:59:59,2023-03-05,372438899,outil,Indemnité de licenciement,view_step,info_cc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
logs.type.value_counts()

visit_content              3195505
outil                      2200818
cc_search                   741430
enterprise_search           670725
cc_search_type_of_users     284588
pagecc_clickcontrib         259535
outlink                     213808
search                      200687
result_candidates           169235
select_related              135119
home                        134428
cc_select_p1                132240
select_result               123009
pagecc_clicktheme           117457
cc_select_traitée           104277
themes                      100089
download                     88787
enterprise_select            65867
pagecc_searchcc              57490
cc_select_p2                 56858
suggestion_candidates        42083
select_suggestion            42082
cc_select_non_traitée        41728
feedback                     32780
cc_select                    31478
modeles_courriers            27545
outils                       18405
next_result_page             14985
contact             

In [8]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [9]:
def clean_dataset_from_useless_rows(df):
    df_shortened = df[df['type'].isin(['visit_content', 'outil', 'themes', 'page_information',
                                       'search', 'home', 'modeles_courriers', 'outlink'])]
    return df_shortened.loc[
        (df_shortened['idVisit'].shift() != df_shortened['idVisit']) |
        (df_shortened['url'].shift() != df_shortened['url']) |
        #(df_shortened['type'].shift() != df_shortened['type']) |
        (df_shortened['referrerTypeName'].shift() != df_shortened['referrerTypeName'])
    ]

In [10]:
logs_formated = format_url_and_get_month_and_datetime_columns(logs.copy())
logs_cleaned = clean_dataset_from_useless_rows(logs_formated.copy())

In [11]:
logs_cleaned.shape[0]

3536807

# Début de l'explo

On va s'y prendre en plusieurs étapes. 
- Supprimons des données tous les visiteurs qui NE sont PAS allés sur les pages info licenciement
- Ordonnons par IdVisit et timestamp nos données
- Débarasser des logs inutiles, nous allons dupliquer la colonne url en une colonne url_précédent en la décalant de 1 ligne (en gros l'url précédent est égal à l'url actuel de la ligne précédente)
    - cela va créer un souci car le premier url de visite d'un utilisateur sera le dernier url de l'utilisateur précédent. On pourra facilement corriger ça en modifiant le premier url_précédent de chaque utilisateur par le referrer_name de la ligne actuelle. De telle sorte à ce que la première ligne contienne le point d'entrée de l'utilisateur et les lignes suivantes soient composé de l'url de la ligne précédente. Est-ce que c'est clair ? 😅 
- Ensuite on voit ce qu'on fait mais on pourrait faire un groupby url et url précédent et count occurences 

### 1. Suppressions des visiteurs qui NE sont PAS allés sur les pages info licenciement

In [13]:
# On priviligie tous les logs pour ne manquer un utilisateur
visitors_having_visited_info_licenc = set(logs[
    (logs['url'].notna()) & 
    (logs['url'].str.contains('information/')) &
    (logs['url'].str.contains('licenciement'))
].idVisit.unique())

len(visitors_having_visited_info_licenc)

4579

In [15]:
logs_visitors_having_visited_info_licenc = logs_cleaned[
    logs_cleaned['idVisit'].isin(visitors_having_visited_info_licenc)
]

### 2. Ordonnons par IdVisit et timestamp nos données

In [16]:
logs_sorted = logs_visitors_having_visited_info_licenc.sort_values(by=['idVisit', 'timestamp'])

### 3. On ajoute une colonne url_précédent

In [28]:
logs_sorted['url_précédent'] = logs_sorted['url'].shift()

In [29]:
logs_sorted.head(5)

,timeSpent,timestamp,url,idVisit,lastActionDateTime,lastActionTimestamp,referrerName,referrerTypeName,serverTimePretty,logfile,uvi,type,outil,outilAction,outilEvent,feedbackType,visited,ccAction,query,idCc,recoSelection,recoType,cc,resultSelection,suggestionPrefix,suggestionSelection,referrerUrl,month,datetime,url_précédent
6797535,0.0,1.678263e+09,https://code.travail.gouv.fr/information/petit...,33736136,2023-03-07 23:02:14,1.678230e+09,Google,Search Engines,00:02:14,2023-03-08,-1.939448e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 09:02:14,None
6792914,20.0,1.678271e+09,https://code.travail.gouv.fr/modeles-de-courri...,33737817,2023-03-08 01:25:13,1.678239e+09,Google,Search Engines,02:25:13,2023-03-08,1.244188e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 11:24:53,https://code.travail.gouv.fr/information/petit...
6792915,0.0,1.678271e+09,https://code.travail.gouv.fr/information/petit...,33737817,2023-03-08 01:25:13,1.678239e+09,Google,Search Engines,02:25:13,2023-03-08,1.244188e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 11:25:13,https://code.travail.gouv.fr/modeles-de-courri...
6790065,5.0,1.678283e+09,https://code.travail.gouv.fr/information/petit...,33738883,2023-03-08 04:36:34,1.678250e+09,Google,Search Engines,05:36:34,2023-03-08,1.613750e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 14:36:06,https://code.travail.gouv.fr/information/petit...
6790066,23.0,1.678283e+09,https://code.travail.gouv.fr/themes/licencieme...,33738883,2023-03-08 04:36:34,1.678250e+09,Google,Search Engines,05:36:34,2023-03-08,1.613750e+09,themes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 14:36:11,https://code.travail.gouv.fr/information/petit...


In [39]:
def update_first_url_precedent_with_refererrName(df):
    df.iloc[0, df.columns.get_loc('url_précédent')] = df.iloc[0].referrerName
    return df

In [41]:
logs_with_previous_url = logs_sorted\
    .groupby(by='idVisit', group_keys=False)\
    .progress_apply(update_first_url_precedent_with_refererrName)

  0%|          | 0/4579 [00:00<?, ?it/s]

,timeSpent,timestamp,url,idVisit,lastActionDateTime,lastActionTimestamp,referrerName,referrerTypeName,serverTimePretty,logfile,uvi,type,outil,outilAction,outilEvent,feedbackType,visited,ccAction,query,idCc,recoSelection,recoType,cc,resultSelection,suggestionPrefix,suggestionSelection,referrerUrl,month,datetime,url_précédent
6797535,0.0,1.678263e+09,https://code.travail.gouv.fr/information/petit...,33736136,2023-03-07 23:02:14,1.678230e+09,Google,Search Engines,00:02:14,2023-03-08,-1.939448e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 09:02:14,Google
6792914,20.0,1.678271e+09,https://code.travail.gouv.fr/modeles-de-courri...,33737817,2023-03-08 01:25:13,1.678239e+09,Google,Search Engines,02:25:13,2023-03-08,1.244188e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 11:24:53,Google
6792915,0.0,1.678271e+09,https://code.travail.gouv.fr/information/petit...,33737817,2023-03-08 01:25:13,1.678239e+09,Google,Search Engines,02:25:13,2023-03-08,1.244188e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 11:25:13,https://code.travail.gouv.fr/modeles-de-courri...
6790065,5.0,1.678283e+09,https://code.travail.gouv.fr/information/petit...,33738883,2023-03-08 04:36:34,1.678250e+09,Google,Search Engines,05:36:34,2023-03-08,1.613750e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 14:36:06,Google
6790066,23.0,1.678283e+09,https://code.travail.gouv.fr/themes/licencieme...,33738883,2023-03-08 04:36:34,1.678250e+09,Google,Search Engines,05:36:34,2023-03-08,1.613750e+09,themes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 14:36:11,https://code.travail.gouv.fr/information/petit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6466633,23.0,1.678256e+09,https://code.travail.gouv.fr/information/grand...,33731789,2023-03-07 21:09:02,1.678223e+09,Google,Search Engines,22:09:02,2023-03-07,-1.171888e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 07:08:15,Google
6466634,24.0,1.678256e+09,https://code.travail.gouv.fr/outils/procedure-...,33731789,2023-03-07 21:09:02,1.678223e+09,Google,Search Engines,22:09:02,2023-03-07,-1.171888e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 07:08:38,https://code.travail.gouv.fr/information/grand...
6466648,0.0,1.678256e+09,https://code.travail.gouv.fr/information/grand...,33731789,2023-03-07 21:09:02,1.678223e+09,Google,Search Engines,22:09:02,2023-03-07,-1.171888e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 07:09:02,https://code.travail.gouv.fr/outils/procedure-...
6442193,802.0,1.678256e+09,https://code.travail.gouv.fr/fiche-ministere-t...,33732048,2023-03-07 21:26:35,1.678224e+09,Google,Search Engines,22:26:35,2023-03-07,-2.009614e+09,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2023-03-08 07:13:13,Google


### 4. Ensuite on voit ce qu'on fait mais on pourrait faire un groupby url et url précédent et count occurences 